In [1]:
from data_sources.clob import CLOBDataSource
from decimal import Decimal

# Get trading rules and candles
clob = CLOBDataSource()

In [6]:
# Constants
CONNECTOR_NAME = "binance"
INTERVAL = "15m"
DAYS = 7
TOP_X_MARKETS = 20  # Number of top markets to select
# Assuming the `top_markets` DataFrame has been generated as shown in the previous steps
# Constants
TOTAL_AMOUNT = 3000  # General total amount for all markets
MIN_AMOUNT_PER_MARKET = 70  # Minimum amount per market
ACTIVATION_BOUNDS = 0.002  # Input activation bounds
MAX_OPEN_ORDERS = 1  # Input max open orders for each market

# Example values
TAKE_PROFIT_MULTIPLIER = 0.1  # Multiplier for take profit based on NATR
AMOUNTS_QUOTE_PCT = [0.1, 0.1, 0.1, 0.1, 0.2, 0.2]  # Weights for each cluster

# Features configuration
VOLATILITY_WINDOW = 100
TREND_SHORT_WINDOW = 20
TREND_LONG_WINDOW = 100
VOLUME_SHORT_WINDOW = 20
VOLUME_LONG_WINDOW = 100


# Download data
- Get trading rules
- Get candles for the last x days

In [7]:
import asyncio

trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset("FDUSD")\
    .filter_by_min_notional_size(Decimal("5"))\
    .get_all_trading_pairs()

tasks = [clob.get_candles_last_days(
    connector_name=CONNECTOR_NAME,
    trading_pair=trading_pair,
    interval=INTERVAL,
    days=DAYS,
) for trading_pair in trading_pairs]

candles = await asyncio.gather(*tasks)
candles = {trading_pair: candle for trading_pair, candle in zip(trading_pairs, candles)}

2024-07-25 20:46:52,852 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x13ece7880>
2024-07-25 20:46:52,853 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x13f583c40>, 717952.677180666)]']
connector: <aiohttp.connector.TCPConnector object at 0x13ece78b0>
2024-07-25 20:46:52,863 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x13f1857b0>
2024-07-25 20:46:52,864 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x13effa1a0>, 717952.695212625)]']
connector: <aiohttp.connector.TCPConnector object at 0x13f1857e0>
2024-07-25 20:46:52,956 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x13f186c50>
2024-07-25 20:46:52,957 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1

In [8]:
from features.candles.trend import TrendConfig
from features.candles.volatility import VolatilityConfig
from features.candles.volume import VolumeConfig
from research_notebooks.dneitor.utils import generate_report

report = generate_report(
    candles=candles,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    trend_config=TrendConfig(short_window=TREND_SHORT_WINDOW, long_window=TREND_LONG_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_SHORT_WINDOW, long_window=VOLUME_LONG_WINDOW))
report

,trading_pair,mean_volatility,mean_natr,mean_bb_width,latest_trend,average_volume_per_hour,normalized_score
0,THETA-FDUSD,0.00456142,0.00359366,7.4168599,0.0725473,2608.26871893,0.00007537
1,TAO-FDUSD,0.00588071,0.00600238,8.71844796,0.25883375,5218.77149466,0.00017741
2,LINK-FDUSD,0.00367386,0.00484175,5.21520546,-0.19775842,45913.8786468,0.00093452
3,PYTH-FDUSD,0.00548008,0.00558894,7.61686157,0.07856553,8609.41210683,0.00025573
4,LDO-FDUSD,0.00540042,0.00533586,7.25688935,-0.20074686,5713.01840077,0.00016168
...,...,...,...,...,...,...,...
87,DOT-FDUSD,0.00325544,0.00344245,4.5893208,0.05139595,10237.32491022,0.00018323
88,APT-FDUSD,0.00429458,0.00283103,5.63447172,0.277841,1359.79582057,0.00002985
89,STRK-FDUSD,0.00621569,0.00632413,8.82312762,-0.02170657,15262.15288435,0.00052511
90,ATOM-FDUSD,0.00328426,0.00323083,4.64793697,0.71754362,6899.21116262,0.00012503


In [9]:
from research_notebooks.dneitor.utils import filter_top_markets

top_markets = filter_top_markets(report_df=report, top_x=TOP_X_MARKETS, min_volume_usd=2000, min_atr=0.008, trend_threshold=-0.5)
top_markets

,trading_pair,mean_volatility,mean_natr,mean_bb_width,latest_trend,average_volume_per_hour,normalized_score
69,PEPE-FDUSD,0.00609313,0.00896949,7.69717123,0.10627382,327655.15545768,0.0098522
85,BANANA-FDUSD,0.01190539,0.01542003,12.97622012,0.12508542,105265.03065414,0.00533651
34,BONK-FDUSD,0.00715163,0.00997337,10.63847693,0.13739111,86350.89134019,0.0035854
79,WIF-FDUSD,0.00731352,0.01061232,9.48835983,-0.04097797,94542.28699686,0.00350366
45,ENA-FDUSD,0.00672577,0.00843915,11.52383822,-0.09436086,67704.51997222,0.00304264
10,ZRO-FDUSD,0.00805923,0.01029048,12.7945335,0.13731974,48196.87274802,0.00240549
52,BOME-FDUSD,0.00583282,0.00829515,8.43463228,-0.04833481,60122.58708807,0.00197959
74,FLOKI-FDUSD,0.0069658,0.00981574,9.08789693,-0.1882034,50135.01179532,0.00177927
5,1000SATS-FDUSD,0.00835836,0.0119137,11.50878076,0.01435414,32272.7205099,0.00145019
21,WLD-FDUSD,0.00687765,0.00976285,9.32006213,0.3748944,29177.29552489,0.00106181


In [10]:
from research_notebooks.dneitor.utils import generate_config, dump_dict_to_yaml

strategy_config = generate_config(
    id="dneitor-binance_0.4",
    connector_name=CONNECTOR_NAME,
    candles=candles,
    top_markets=top_markets,
    total_amount=TOTAL_AMOUNT,
    amounts_quote_pct=AMOUNTS_QUOTE_PCT,
    activation_bounds=ACTIVATION_BOUNDS,
    take_profit_multiplier=TAKE_PROFIT_MULTIPLIER,
    max_open_orders=MAX_OPEN_ORDERS,
    min_amount_per_market=MIN_AMOUNT_PER_MARKET,
)
dump_dict_to_yaml("configs/", strategy_config)